# Data Cleanup and Quality Check

**Author:** Diwas Puri (diwas.puri@duke.edu), Duke University

This notebook performs data cleaning on the scraped YBIO dataset.

**Input:** `../data/organizations_final.csv`  
**Output:** `../data/organizations_clean.csv`

In [ ]:
import pandas as pd
import numpy as np
import os
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
# plotting code here
warnings.filterwarnings("default", category=FutureWarning)


# Define paths
INPUT_FILE = '../data/organizations_final.csv'
OUTPUT_FILE = '../data/organizations_clean.csv'

# Load data
print(f"Loading data from {INPUT_FILE}...")
df = pd.read_csv(INPUT_FILE)

initial_count = len(df)
print(f"Initial row count: {initial_count:,}")
display(df.head())

Loading data from ../data/organizations_final.csv...
Initial row count: 78,865


,Name,Acronym,Founded,City,Country,Type I,Type II,UID
0,Federation of International Dance Festivals,FIDAF,2012.0,Chungnam,Korea Rep,B,NaN,XJ9650
1,Federation of International Geo-Engineering So...,FedIGS,2007.0,NaN,NaN,B,y,AA0037
2,FIDI Global Alliance,NaN,1950.0,Zaventem,Belgium,B,NaN,XB0947
3,Food and Agriculture Organization of the Unite...,FAO,1945.0,Rome,Italy,B,g,XB0971
4,Gender and Water Alliance,GWA,2000.0,Dieren,Netherlands,B,NaN,XM0973


## 1. Remove Exact Duplicates
We remove rows that are identical across all columns.

In [2]:
df_deduped = df.drop_duplicates()
deduped_count = len(df_deduped)
print(f"Rows after removing exact duplicates: {deduped_count:,}")
print(f"Removed: {initial_count - deduped_count:,} rows")

Rows after removing exact duplicates: 78,865
Removed: 0 rows


## 2. Remove Repeated Headers
Sometimes scraping artifacts include the header row in the data body.

In [3]:
if 'Name' in df_deduped.columns:
    header_rows = df_deduped[df_deduped['Name'] == 'Name']
    if not header_rows.empty:
        print(f"Found {len(header_rows)} repeated header rows. Removing...")
        df_deduped = df_deduped[df_deduped['Name'] != 'Name']
    else:
        print("No repeated header rows found.")

No repeated header rows found.


## 3. Check for Duplicate UIDs
Ensure each organization has a unique identifier.

In [4]:
if 'UID' in df_deduped.columns:
    uids = df_deduped['UID']
    dup_uids = uids[uids.duplicated()]
    if not dup_uids.empty:
        print(f"Found {len(dup_uids)} duplicate UIDs.")
        # Keep the first occurrence
        df_deduped = df_deduped.drop_duplicates(subset=['UID'], keep='first')
        print(f"Rows after removing duplicate UIDs: {len(df_deduped):,}")
    else:
        print("No duplicate UIDs found.")

No duplicate UIDs found.


## 4. Standardize Missing Values
Convert empty strings and whitespace to NaN.

In [5]:
df_deduped = df_deduped.replace(r'^\s*$', np.nan, regex=True)
print("Standardized missing values.")

Standardized missing values.


## 5. Save Cleaned Data

In [6]:
print(f"Final row count: {len(df_deduped):,}")
df_deduped.to_csv(OUTPUT_FILE, index=False)
print(f"Cleaned data saved to: {OUTPUT_FILE}")

Final row count: 78,865
Cleaned data saved to: ../data/organizations_clean.csv
